# Features Constantes y Quasi-Constantes

En el presente cuaderno se explora cómo manejar features constantes y quasi-constantes en un data set. Dicho manejo se realiza mediante el uso de herramientas de Scikit-learn, así como manualmente.

---
## Features constantes

Estos son aquellos features cuyo valor no cambia. Puede visualizarse como una columna que tiene siempre el mismo valor para todas sus filas. Evidentemente, un feature de este tipo no brinda información valiosa para realizar predicciones con un modelo de machine learning, razón por la cual se eliminan y no se toman en cuenta.

A continuación, se muestra cómo identificar y eliminar features constantes utilizando VarianceThreshold de Scikit-learn y utilizando código normal.

In [57]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

data = pd.read_csv('notebooks/dataset_1.csv')

### VarianceThreshold

Mostramos primero cómo trabajar con VarianceThreshold. Es importante resaltar que este método solo sirve para variables numéricas.

**Importante:** el estudio para la selección de variables debería realizarse solo sobre el conjunto de train con el objetivo de evitar overfitting, pero para efectos de este resumen, se utilizará todo el dataset.

In [58]:
# se seleccionan solo las variables cuya varianza sea mayor a 0
sel = VarianceThreshold(threshold=0) # especifica que varianza debe ser mayor a 0
sel.fit(data)  # busca las variables cuya varianza es 0
print(sel.get_support()) # muestra cuáles variables se tomarán en cuenta (True)
print(sum(sel.get_support())) # muestra cuántas variables se tomarán en cuenta

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False  True  True  True
  True  True False  True False  True  True False  True  True  True  True
 False  True False  True  True  True  True  True  True  True  True  True
  True  True  True False False  True  True  True  True  True  True False
  True False  True  True  True  True False  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  T

In [59]:
# escogemos las columnas que son constantes 
constant = data.columns[~sel.get_support()] # nótese el uso de ~ para indicar que queremos las que no son constantes
print("Variables constantes:", len(constant), "\n") # también: print(sum(~sel.get_support()))

# incluso podemos visualizar que cada una de estas columnas solo tiene un valor repetido
for col in constant:
    print(col, data[col].unique())

Variables constantes: 33 

var_23 [0]
var_33 [0]
var_44 [0]
var_61 [0]
var_80 [0]
var_81 [0]
var_87 [0]
var_89 [0.]
var_92 [0]
var_97 [0]
var_99 [0]
var_112 [0]
var_113 [0]
var_120 [0]
var_122 [0]
var_127 [0]
var_135 [0]
var_158 [0]
var_167 [0]
var_171 [0]
var_178 [0.]
var_180 [0.]
var_182 [0]
var_195 [0]
var_196 [0]
var_201 [0]
var_212 [0]
var_215 [0]
var_225 [0]
var_227 [0.]
var_248 [0]
var_294 [0]
var_297 [0]


In [60]:
# VarianceThreshold retorna un numpy array, entonces hay que guardar el nombre de las columnas 
# para poder reconstruir un DataFrame
columns = data.columns[sel.get_support()]

data = sel.transform(data) # en esta etapa, se transformaría el train y el test
data = pd.DataFrame(data, columns=columns)
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_290,var_291,var_292,var_293,var_295,var_296,var_298,var_299,var_300,target
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
1,0.0,0.0,0.0,3.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
2,0.0,0.0,0.0,5.88,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,67772.7216,0.0
3,0.0,0.0,0.0,14.10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
4,0.0,0.0,0.0,5.76,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0


### Método manual para variables numéricas

Al igual que con VarianceThreshold, realizaremos las operaciones sobre todo el dataset, pues lo que interesa es mostrar los métodos. No obstante, en un caso real, se debería separar en conjuntos de train y test y realizar el análisis pre-selección sobre el conjunto de train.

In [61]:
# utilizando std() de pandas, seleccionamos las columnas cuya varianza sea 0
constant_cols = [col for col in data.columns if data[col].std() == 0]

# eliminamos todas las columnas constantes
# en un caso real, el drop se aplica tanto a train como a test
data.drop(labels=constant_cols, axis=1, inplace=True)
print(len(data.columns))

268


### Método manual para variables categóricas

Si bien se podrían convertir las variables categóricas a numéricas para luego aplicar alguno de los métodos anteriores, esto implicaría tiempo de preprocesamiento que podría ser costoso en variables que de por sí no se van a utilizar.

A continuación se plantea una mejor alternativa, teniendo en cuenta la consideración sobre analizar solo los datos del conjunto de train.

In [62]:
# Puesto que el dataset que estamos estudiando es completamente numérico
# convertimos las variables a objetos para simular que son categóricas
data = data.astype('O')
print(data.dtypes.head())

# utilizando nunique() de pandas, seleccionamos las columnas cuya solo tengan 1 valor
# dado que nunique() retorna el número de valores distintos
# se puede considerar nunique(dropna=False) si hay valores nulos, pues el método los ignora por default
constant_cols = [col for col in data.columns if data[col].nunique() == 1]

# eliminamos todas las columnas constantes
# en un caso real, el drop se aplica tanto a train como a test
data.drop(labels=constant_cols, axis=1, inplace=True)
print(len(data))

var_1    object
var_2    object
var_3    object
var_4    object
var_5    object
dtype: object
50000


---
## Features Quasi-Constantes

Estos son aquellos features cuyo valor *casi* no cambia. Puede visualizarse como una columna que tiene el mismo valor para la gran mayoría de sus filas. Generalmente un feature de este tipo no brinda información valiosa para realizar predicciones con un modelo de machine learning, razón por la cual generalmente se eliminan y no se toman en cuenta.

A continuación, se muestra cómo identificar y eliminar features quasi-constantes utilizando VarianceThreshold de Scikit-learn y utilizando código manual.

In [63]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

data = pd.read_csv('notebooks/dataset_1.csv')

# Se eliminan los valores constantes
constant_features = [
    feature for feature in data.columns if data[feature].std() == 0
]
data.drop(labels=constant_features, axis=1, inplace=True)

# Copia para ejemplo manual
#data_1 = data.copy()
data_1 = data

### VarianceThreshold

Mostramos primero cómo trabajar con VarianceThreshold. Es importante resaltar que este método solo sirve para variables numéricas.

**Importante:** el estudio para la selección de variables debería realizarse solo sobre el conjunto de train con el objetivo de evitar overfitting, pero para efectos de este resumen, se utilizará todo el dataset.

In [64]:
# Se seleccionan solo las variables cuya varianza sea mayor a 0.01
sel = VarianceThreshold(threshold=0.01) # especifica que varianza debe ser mayor a 0.01
sel.fit(data)  # busca las variables cuya varianza es mayor a 0.01

#print(sel.get_support()) # muestra cuáles variables se tomarán en cuenta (True)

print("Número de variables que se conservan: ", sum(sel.get_support())) # muestra cuántas variables se tomarán en cuenta

# Se toman las columnas constantes
quasi_constant = data.columns[~sel.get_support()] # nótese el uso de ~ para indicar que queremos las que no son constantes
not_quasi_constant = data.columns[sel.get_support()] 

print("Número de variables que se eliminan: ", len(quasi_constant)) # muestra cuántas variables se tomarán en cuenta
print()
print("Variables que se eliminan: ")
print(quasi_constant)


Número de variables que se conservan:  219
Número de variables que se eliminan:  49

Variables que se eliminan: 
Index(['var_2', 'var_7', 'var_9', 'var_10', 'var_19', 'var_28', 'var_36',
       'var_43', 'var_45', 'var_53', 'var_56', 'var_59', 'var_66', 'var_67',
       'var_69', 'var_71', 'var_104', 'var_106', 'var_116', 'var_133',
       'var_137', 'var_141', 'var_170', 'var_177', 'var_187', 'var_189',
       'var_194', 'var_197', 'var_198', 'var_218', 'var_219', 'var_223',
       'var_233', 'var_234', 'var_235', 'var_245', 'var_247', 'var_249',
       'var_250', 'var_251', 'var_256', 'var_260', 'var_267', 'var_274',
       'var_282', 'var_285', 'var_287', 'var_289', 'var_298'],
      dtype='object')


A continuación se muestran los porcentajes de observaciones de cada valor que pueden tomar variables quasi-constantes. En este caso se utilizan las tres primeras variables.

In [65]:
# Analizando algunas variables quasi-constantes
# Procentajes de observaciones de cada valor de la variable
print("Variables quasi-constantes: ")
print()
for i in range(3):
    print("Porcentajes de observaciones por valor para la variable:", quasi_constant[i])
    print(data[quasi_constant[i]].value_counts() / float(len(data)))

Variables quasi-constantes: 

Porcentajes de observaciones por valor para la variable: var_2
0    0.99994
1    0.00006
Name: var_2, dtype: float64
Porcentajes de observaciones por valor para la variable: var_7
0    0.9999
3    0.0001
Name: var_7, dtype: float64
Porcentajes de observaciones por valor para la variable: var_9
0    0.99992
3    0.00008
Name: var_9, dtype: float64


Cómo se puede apreciar se tienen porcentajes muy altos de un valor especifico pero no es el único valor que toma la variable.

A continuación se mostrarán los mismos procentajes para una variable que se mantiene.

In [66]:
print("Variables que se mantienen: ")
print()
for i in range(2):
    print("Porcentajes de observaciones por valor para la variable:", not_quasi_constant[i])
    print(data[not_quasi_constant[i]].value_counts() / float(len(data)))

Variables que se mantienen: 

Porcentajes de observaciones por valor para la variable: var_1
0    0.99950
3    0.00030
6    0.00016
9    0.00004
Name: var_1, dtype: float64
Porcentajes de observaciones por valor para la variable: var_3
0.0000         0.99950
121142.0394    0.00002
861.0900       0.00002
6211.5165      0.00002
7134.8904      0.00002
207901.3365    0.00002
10308.2616     0.00002
2641.0164      0.00002
12644.1000     0.00002
10281.6000     0.00002
10385.4912     0.00002
5194.1709      0.00002
12542.3100     0.00002
101195.4735    0.00002
52105.7901     0.00002
86718.0000     0.00002
5209.9500      0.00002
14629.9626     0.00002
25905.4866     0.00002
3583.3941      0.00002
35685.9459     0.00002
13297.0320     0.00002
2928.9150      0.00002
15028.0560     0.00002
27.3000        0.00002
16086.9720     0.00002
Name: var_3, dtype: float64


En este caso se aprecia que algunas de las variables que se mantienen, al igual que en las quasi-constantes, se presentan porcentajes muy altos de observación de un valor especifico. La diferencia en este caso con las variables marcadas cómo quasi-constantes es que existe más variedad de valores, lo que altera la varianza de la variable.

In [67]:
# VarianceThreshold retorna un numpy array, entonces hay que guardar el nombre de las columnas 
# para poder reconstruir un DataFrame
columns = data.columns[sel.get_support()]

data = sel.transform(data) # en esta etapa, se transformaría el train y el test
data = pd.DataFrame(data, columns=columns)
data.head()

,var_1,var_3,var_4,var_5,var_6,var_8,var_11,var_12,var_13,var_14,...,var_288,var_290,var_291,var_292,var_293,var_295,var_296,var_299,var_300,target
0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
1,0.0,0.0,3.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
2,0.0,0.0,5.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,67772.7216,0.0
3,0.0,0.0,14.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
4,0.0,0.0,5.76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0


### Método manual para variables numéricas

Al igual que con VarianceThreshold, realizaremos las operaciones sobre todo el dataset, pues lo que interesa es mostrar los métodos. No obstante, en un caso real, se debería separar en conjuntos de train y test y realizar el análisis pre-selección sobre el conjunto de train.

In [68]:
# Lista de variables quasi-constantes
quasi_constant_features = []

for feature in data_1.columns:

    # Se busca el porcentaje de observación del valor predominante en cada columna
    
    predominant = (data_1[feature].value_counts() / float(
        len(data_1))).sort_values(ascending=False).values[0]

    # Si el porcentaje del valor predominante es mayor a 0.998 se toma cómo quasi-constante
    if predominant > 0.998:
        quasi_constant_features.append(feature)

print("Número de variables quasi-constantes:",len(quasi_constant_features))

print("Variables quasi-constantes: ", quasi_constant_features )


Número de variables quasi-constantes: 104
Variables quasi-constantes:  ['var_1', 'var_2', 'var_3', 'var_6', 'var_7', 'var_9', 'var_10', 'var_11', 'var_12', 'var_14', 'var_20', 'var_24', 'var_28', 'var_32', 'var_34', 'var_36', 'var_39', 'var_40', 'var_42', 'var_43', 'var_45', 'var_48', 'var_53', 'var_59', 'var_60', 'var_65', 'var_66', 'var_67', 'var_69', 'var_71', 'var_72', 'var_73', 'var_77', 'var_78', 'var_90', 'var_95', 'var_98', 'var_102', 'var_104', 'var_106', 'var_111', 'var_115', 'var_116', 'var_124', 'var_125', 'var_126', 'var_129', 'var_133', 'var_136', 'var_138', 'var_141', 'var_142', 'var_146', 'var_149', 'var_150', 'var_151', 'var_153', 'var_159', 'var_170', 'var_183', 'var_184', 'var_187', 'var_189', 'var_197', 'var_202', 'var_204', 'var_210', 'var_211', 'var_216', 'var_217', 'var_219', 'var_221', 'var_223', 'var_224', 'var_228', 'var_233', 'var_234', 'var_235', 'var_236', 'var_237', 'var_239', 'var_243', 'var_245', 'var_246', 'var_247', 'var_249', 'var_254', 'var_257', 'va

En este caso el método manual encontró más variables quasi-constantes.
Esto se debe a que el método manual utiliza unicamente el porcentaje de apariciones y no la varianza cómo VarianceThreshold.

Por esta razón es que muchas variables que tenían más variedad de valores, pero con bajas observaciones, en este caso sí se consideran quasi-constantes. Un ejemplo de esto es la variable "var_3" vista anteriormente. 

In [70]:
# Se eliminan las variables quasi-constantes
data_1.drop(labels=quasi_constant_features, axis=1, inplace=True)

print("Resultado con método VarianceThreshold: ", data.shape)
print("Resultado con método manual: ", data_1.shape)

data_1.head()

Resultado con método VarianceThreshold:  (50000, 219)
Resultado con método manual:  (50000, 164)


,var_4,var_5,var_8,var_13,var_15,var_16,var_17,var_18,var_19,var_21,...,var_281,var_284,var_288,var_291,var_292,var_293,var_295,var_296,var_300,target
0,0.00,0.0,0,0.0,0,0.0,0.0,0.00,0,0.0000,...,0,0,0.0,0,0.0,0,0,0,0.0000,0
1,3.00,0.0,0,0.0,3,0.0,0.0,0.00,0,2.7300,...,0,0,0.0,0,0.0,0,0,0,0.0000,0
2,5.88,0.0,0,0.0,3,0.0,0.0,0.00,0,19.8990,...,0,1,0.0,0,0.0,0,3,0,67772.7216,0
3,14.10,0.0,0,0.0,0,0.0,0.0,988.47,0,0.0000,...,0,0,0.0,0,0.0,0,0,0,0.0000,0
4,5.76,0.0,0,0.0,3,0.0,0.0,0.00,0,5981.1741,...,0,0,0.0,0,0.0,0,0,0,0.0000,0
